# Exploratory Data Analysis of Electric Vehicle Population Data

- **Course:** CS660/71425 Mathematical Foundations of Analytics
- **Instructor:** Prof. Sarbanes
- **Group-1:** Will Torres, Mike Griffin, Watson Blair, Syed Abdul Mubashir, Mohammed Abdul Munaf
- **Semester:** Fall 2024
- **Project #:** 1
- **Due Date:** 07-Oct-2024

## Project Description
Exploratory Data Analysis (EDA) is essential for understanding, cleaning, and preparing data for further analysis in data science projects. This project focuses on analyzing the Electric Vehicle (EV) population dataset from Washington State, USA.


## Questions to be Answered
1. Which car manufacturers are most commonly used for EVs in Washington?
2. What are the highest and lowest electric ranges in this dataset, and which car makers and models do they correspond to?
3. Is the maximum electric range value unique? If not, which cars share this range?
4. Is the minimum electric range value unique? If not, which cars share this range?
5. How does the electric range vary between car makers and between models?
6. Which are the top 5 cities adopting EVs?
7. How does the EV adoption rate vary among car makers over the years?
8. Is there a correlation between the electric range and the city of an EV?
9. Which county has the greatest variety of EV car models?

## EDA

### Step 1: Understand the Dataset Context

_Washington State Department of Licensing_'s dataset tracks **Battery Electric Vehicles** (BEVs) and **Plug-in Hybrid Electric Vehicles** (PHEVs) registered in Washington state. This dataset is updated monthly and offers insights into the electric vehicle population, categorized by county, postal code, and vehicle characteristics.

**Key Dataset Details**:
- Data Source: Washington State Department of Licensing.
- Objective: To provide a comprehensive overview of electric vehicles registered in Washington, segmented by various geographical and vehicle-specific metrics. It helps track the adoption of electric vehicles across different regions.
- Metadata: The dataset is updated regularly (most recently on September 16, 2024) and covers registered electric vehicles as of August 31, 2024.  

**Notable Fields**:
- VIN (1-10): The first 10 characters of the Vehicle Identification Number.
- County/**City**/State/Postal Code: Geographic data identifying where the vehicle is registered.
- **Model Year**, **Make**, **Model**: Vehicle characteristics.
- Electric Vehicle Type: Whether the vehicle is a BEV or PHEV.
- **Electric Range**: The distance a vehicle can travel on electric charge.
- Legislative District: The political district in which the vehicle owner resides.  

**Objective Clarification**:
- The goal of using this data could be to analyze electric vehicle adoption trends in Washington, identify regions with the highest EV adoption, or track specific vehicle models.

### Step 2: Import Libraries and Load Data
- **Import Necessary Libraries:** `pandas`, `numpy`, `matplotlib`, `seaborn`
- **Load the Dataset:** `EV_Population_WA_Data.csv`

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Load the dataset
df = pd.read_csv('data/EV_Population_WA_Data.csv')


"""Load Data"""
rawData = pd.read_csv('./data/EV_Population_WA_Data.csv')

### Step 3: Initial Data Inspection
- **View Data Structure:** `.head()`, `.info()`, `.describe()`
- **Check Dimensions:** `.shape()`
- **Identify Missing Values:** `.isnull().sum()`

In [ ]:
df.head(10)

In [ ]:
"""
from the non-null counts we can identify the presence of some outliers in our data.
Further investigation reveals at 4 of these Vehicles appear to be in Canada (their State info designates BC)
An additional 4 Vehicles are missing geolocation data,

"""
df.info()

In [ ]:
df.describe()

In [ ]:
print(f"Dataset Shape: {df.shape[0]} rows, {df.shape[1]} cols")

In [ ]:
df.isnull().sum().sort_values(ascending = False)

### Step 4: Data Cleaning
- **Handle Missing/Incomplete Data**
  - Range Data
- **Handle Outliers**
- **Correct Data Types**
  - transform categorical data into numeric values for use in correlation operations 
  - 
- **Handle Duplicates**

In [15]:
from utils import calculateRange, calculateMSRP, convertEligibility, purgeInternationalOutliers, convertPostalCode
# , imputeLocationData
rawData = df
# Note: The questions pertain to the following columns:
# Model Year, Model, Make, Electric Range, City, County
# Focusing our data cleaning, prep, analysis, and feature engineering to these cols might be beneficial.

# Other cleaning:
# - Standardizing Electric Utility, Dropping Duplicates + Missing Vals, imputing values in ER (and MSRP), etc.


cleanData = rawData.copy(deep=True)

cleanData = calculateRange(cleanData)

cleanData = calculateMSRP(cleanData) # Corrects aprox 10,000 records

cleanData = convertEligibility(cleanData)

cleanData = purgeInternationalOutliers(cleanData)

cleanData = convertPostalCode(cleanData)

cleanData = imputeLocationData(cleanData)



In [ ]:
# print(cleanData.head())

# print(rawData[rawData['Vehicle Location'].isna() & rawData['State'].isin(['WA'])])
cleanData.info()

### Step 5: Univariate Analysis
- **Summary Statistics**
- **Visualize Distributions:** histograms, box plots, bar charts

In [ ]:
# Summary Statistics for categorical columns
categorical_summary = df.describe(include=['object'])
categorical_summary

In [ ]:
# Visualizing distributions for numerical variables
num_cols = ['Electric Range', 'Base MSRP', 'Model Year']

for col in num_cols:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[col], kde=True, bins=20)
    plt.title(f'Histogram for {col}')
    plt.show()

# Box plots for numerical variables
for col in num_cols:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=df[col])
    plt.title(f'Box Plot for {col}')
    plt.show()

In [ ]:
# Top 5 Cities with Most Electric Vehicles
def plot_top_5_cities(df):
    top_5_cities = df['City'].value_counts().nlargest(5)
    plt.figure(figsize=(10, 6))
    sns.barplot(x=top_5_cities.values, y=top_5_cities.index, palette="viridis")
    plt.title('Top 5 Cities with Most Electric Vehicles')
    plt.xlabel('Number of Electric Vehicles')
    plt.ylabel('City')
    plt.show()

# Electric Vehicle Types Distribution by State (for Washington 'WA')
def plot_ev_type_by_state(df, state='WA'):
    plt.figure(figsize=(12, 6))
    sns.countplot(data=df[df.State == state], x='State', hue='Electric Vehicle Type', palette="viridis")
    plt.title(f'Electric Vehicle Types Distribution in {state}')
    plt.xlabel('State')
    plt.ylabel('Count of Electric Vehicles')
    plt.xticks(rotation=90)
    plt.legend(title='Electric Vehicle Type')
    plt.show()

# Top 10 and Bottom 10 Makes and Models Subplots
def plot_top_bottom_makes_models(df):
    # Data preparation for top and bottom makes and models
    top_10_makes = df['Make'].value_counts().nlargest(10)
    bottom_10_makes = df['Make'].value_counts().nsmallest(10)
    top_10_models = df['Model'].value_counts().nlargest(10)
    bottom_10_models = df['Model'].value_counts().nsmallest(10)

    # Create subplots: 2 rows and 2 columns
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))

    # Top 10 Makes
    sns.barplot(x=top_10_makes.values, y=top_10_makes.index, ax=axes[0, 0], palette="viridis")
    axes[0, 0].set_title('Top 10 Makes of Electric Vehicles')
    axes[0, 0].set_xlabel('Number of Electric Vehicles')
    axes[0, 0].set_ylabel('Make')

    # Bottom 10 Makes
    sns.barplot(x=bottom_10_makes.values, y=bottom_10_makes.index, ax=axes[0, 1], palette="viridis")
    axes[0, 1].set_title('Bottom 10 Makes of Electric Vehicles')
    axes[0, 1].set_xlabel('Number of Electric Vehicles')
    axes[0, 1].set_ylabel('Make')

    # Top 10 Models
    sns.barplot(x=top_10_models.values, y=top_10_models.index, ax=axes[1, 0], palette="viridis")
    axes[1, 0].set_title('Top 10 Electric Vehicle Models')
    axes[1, 0].set_xlabel('Number of Electric Vehicles')
    axes[1, 0].set_ylabel('Model')

    # Bottom 10 Models
    sns.barplot(x=bottom_10_models.values, y=bottom_10_models.index, ax=axes[1, 1], palette="viridis")
    axes[1, 1].set_title('Bottom 10 Electric Vehicle Models')
    axes[1, 1].set_xlabel('Number of Electric Vehicles')
    axes[1, 1].set_ylabel('Model')

    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

# Call functions with df
plot_top_5_cities(df)
plot_ev_type_by_state(df, state='WA')
plot_top_bottom_makes_models(df)

### Step 6: Bivariate Analysis
- **Correlation Analysis**: (e.g., Pearson, Spearman)
- **Cross-tabulation**
- **Visualize Relationships:** scatter plots, box plots, heatmaps

In [ ]:
# Select only numeric columns for correlation analysis
numeric_cols = df.select_dtypes(include=[np.number])

# Pearson Correlation Matrix
pearson_corr = numeric_cols.corr(method='pearson')

# Spearman Correlation Matrix
spearman_corr = numeric_cols.corr(method='spearman')

# Visualize Pearson Correlation Matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm')
plt.title('Pearson Correlation Matrix')
plt.show()

# Visualize Spearman Correlation Matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm')
plt.title('Spearman Correlation Matrix')
plt.show()

correlation_matrix = df[['Electric Range', 'Base MSRP', 'Model Year']].corr(method='pearson')

# Heatmap for correlations
sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Matrix')
plt.show()

# Cross-tabulation between Electric Vehicle Type and Clean Alternative Fuel Vehicle (CAFV) Eligibility
crosstab_result = pd.crosstab(df['Electric Vehicle Type'], df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'])

# Visualize cross-tabulation using a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(crosstab_result, annot=True, cmap='Blues')
plt.title('Cross-tabulation between Electric Vehicle Type and CAFV Eligibility')
plt.show()

# Scatter plot between Electric Range and Base MSRP
sns.scatterplot(x='Electric Range', y='Base MSRP', data=df)
plt.title('Electric Range vs. Base MSRP')
plt.show()

# Scatter plot ER and Model Year
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Model Year', y='Electric Range', hue='Make', style='Make')
plt.title('Scatter Plot of Electric Range vs Model Year')
plt.xlabel('Model Year')
plt.ylabel('Electric Range')
plt.legend(title='Make', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Boxplot of Electric Range across Makes
sns.boxplot(x='Make', y='Electric Range', data=df)
plt.xticks(rotation=90)
plt.title('Electric Range by Car Maker')
plt.show()


### Step 7: Multivariate Analysis
- **Pairplot/Scatterplot Matrix**
- **Multivariate Statistics**
- **Advanced Visualizations**

In [ ]:
sns.pairplot(df, hue='Make')
plt.title('Pair Plot of Electric Vehicle Data')
plt.show()

In [ ]:
stacked_data = df.groupby(['Model Year', 'Make']).size().unstack()
stacked_data.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Number of EVs by Make and Model Year')
plt.xlabel('Model Year')
plt.ylabel('Number of EVs')
plt.legend(title='Make', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

### Step 8: Feature Engineering
- **Create New Features**
- **Feature Transformation**
- **Encoding Categorical Variables**

Suggestion for New Feature:
- Average ER on the City, County, Make/Model level
- Age of Vehicle
- Create a new feature that combines Model Year and Make to see if certain manufacturers are more popular in specific years.
- Create a feature that combines City and Make to see which manufacturers are dominant in specific cities.
- Range Category:
    - Create categorical bins for Electric Range to identify different range segments (e.g., Low, Medium, High).
    - Normalized Electric Range: Scale the electric range to a 0-1 range using Min-Max normalization.

### Step 9: Handle Imbalanced Data (If Applicable)
- **Resampling Techniques**: Use oversampling, under-sampling, or SMOTE if the target variable is imbalanced

### Step 10: Analyze and Validate Assumptions
- **Check for Multicollinearity**: Use VIF (Variance Inflation Factor) to detect multicollinearity among predictors.
- **Normality Testing**: Test if numerical data follows a normal distribution (e.g., using the Shapiro-Wilk test).
- **Homoscedasticity**: Check the equality of variance across groups

### Step 11: Preliminary Insights and Hypotheses
- **Identify Key Findings**
- **Generate Hypotheses**

In [ ]:
### 1. **Most Common Car Manufacturers for EVs in Washington**:
#   - Use `value_counts()` on the **Make** column to find the most frequent car manufacturers.
df['Make'].value_counts().head(10)

### 2. **Highest and Lowest Electric Ranges and Their Car Makers/Models**:
#   - Find the maximum and minimum values of the **Electric Range** and identify the corresponding car models.
highest_range = df['Electric Range'].max()
lowest_range = df['Electric Range'].min()

max_range_car = df[df['Electric Range'] == highest_range][['Make', 'Model']]
min_range_car = df[df['Electric Range'] == lowest_range][['Make', 'Model']]
print(max_range_car, min_range_car)

### 3. **Is the Maximum Electric Range Unique?**:
#   - Filter the dataset to check how many vehicles have the maximum electric range.
cars_with_max_range = df[df['Electric Range'] == highest_range]
cars_with_max_range

### 4. **Is the Minimum Electric Range Unique?**:
#   - Similar to the maximum range, filter to check how many cars have the minimum electric range.
cars_with_min_range = df[df['Electric Range'] == lowest_range]
cars_with_min_range

### 5. **Variation in Electric Range Between Car Makers and Models**:
#   - Use `groupby()` to calculate the average electric range by car maker and model.
range_by_maker_model = df.groupby(['Make', 'Model'])['Electric Range'].mean().reset_index()
range_by_maker_model

### 6. **Top 5 Cities Adopting EVs**:
#   - Use `value_counts()` on the **City** column to find the top 5 cities with the most EVs.
top_cities = df['City'].value_counts().head(5)
top_cities

### 7. **EV Adoption Rate by Car Makers Over the Years**:
#   - Use `groupby()` on **Make** and **Model Year** to track how EV adoption has changed over time.
adoption_rate_by_maker_year = df.groupby(['Make', 'Model Year']).size().reset_index(name='Count')
adoption_rate_by_maker_year

### 8. **Correlation Between Electric Range and City**:
#   - This can be checked using correlation analysis between **Electric Range** and **City**.
#   - Since cities are categorical, you may need to encode the cities and then calculate correlation.

# [Result]

### 9. **County with the Greatest Variety of EV Models**:
variety_by_county = df.groupby('County')['Model'].nunique().reset_index(name='Unique Models')
top_county = variety_by_county.sort_values(by='Unique Models', ascending=False).head(1)
top_county

### Step 12: Document and Communicate Findings
- **Create Visual Summaries**
- **Write a Summary Report**

### Step 13: Next Steps
- **Plan for Further Analysis**

### Step 14: Review and Reiterate
- **Review EDA**
- **Iterate as Needed**